In [ ]:
pip install -q sentence-transformers faiss-cpu pillow transformers git+https://github.com/openai/whisper.git PyPDF2


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.2 MB/s eta 0:00:00


In [ ]:
from google.colab import files
files.upload()


Saving sample_document.pdf to sample_document.pdf
Saving sample_audio.wav to sample_audio.wav
Saving sample_image.jpg to sample_image.jpg
Saving sample_text.txt to sample_text (1).txt


{'sample_document.pdf': b'%PDF-1.4\n%\x93\x8c\x8b\x9e ReportLab Generated PDF document http://www.reportlab.com\n1 0 obj\n<<\n/F1 2 0 R\n>>\nendobj\n2 0 obj\n<<\n/BaseFont /Helvetica /Encoding /WinAnsiEncoding /Name /F1 /Subtype /Type1 /Type /Font\n>>\nendobj\n3 0 obj\n<<\n/Contents 7 0 R /MediaBox [ 0 0 595.2756 841.8898 ] /Parent 6 0 R /Resources <<\n/Font 1 0 R /ProcSet [ /PDF /Text /ImageB /ImageC /ImageI ]\n>> /Rotate 0 /Trans <<\n\n>> \n  /Type /Page\n>>\nendobj\n4 0 obj\n<<\n/PageMode /UseNone /Pages 6 0 R /Type /Catalog\n>>\nendobj\n5 0 obj\n<<\n/Author (\\(anonymous\\)) /CreationDate (D:20260104164721+00\'00\') /Creator (\\(unspecified\\)) /Keywords () /ModDate (D:20260104164721+00\'00\') /Producer (ReportLab PDF Library - www.reportlab.com) \n  /Subject (\\(unspecified\\)) /Title (\\(anonymous\\)) /Trapped /False\n>>\nendobj\n6 0 obj\n<<\n/Count 1 /Kids [ 3 0 R ] /Type /Pages\n>>\nendobj\n7 0 obj\n<<\n/Filter [ /ASCII85Decode /FlateDecode ] /Length 191\n>>\nstream\nGappWd0bIj

In [ ]:
# ========== IMPORTS ==========
import os
import faiss
import numpy as np
import torch
import whisper
from PIL import Image
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer
from transformers import CLIPProcessor, CLIPModel

# ========== MODELS ==========
text_model = SentenceTransformer("all-MiniLM-L6-v2")
whisper_model = whisper.load_model("base")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# ========== FAISS ==========
dimension = 384
index = faiss.IndexFlatL2(dimension)
documents = []

def add_text(text, source):
    vec = text_model.encode([text]).astype("float32")
    index.add(vec)
    documents.append(f"{source}: {text[:80]}")

# ========== TEXT FILE ==========
with open("/content/sample_text (1).txt", "r") as f:
    add_text(f.read(), "Text file")

# ========== PDF FILE ==========
reader = PdfReader("/content/sample_document.pdf")
pdf_text = " ".join(page.extract_text() or "" for page in reader.pages)
add_text(pdf_text, "PDF file")

# ========== AUDIO FILE ==========
audio_text = whisper_model.transcribe("/content/sample_audio.wav")["text"]
add_text(audio_text, "Audio file")

# ========== IMAGE FILE ==========
image = Image.open("/content/sample_image.jpg")
inputs = clip_processor(images=image, return_tensors="pt")

with torch.no_grad():
    img_vec = clip_model.get_image_features(**inputs)

img_vec = img_vec.numpy()[:, :dimension].astype("float32")
index.add(img_vec)
documents.append("Image file: visual content")

print("✅ Total vectors stored:", index.ntotal)

# ========== SEARCH ==========
def search(query, k=3):
    qvec = text_model.encode([query]).astype("float32")
    _, idx = index.search(qvec, k)
    return [documents[i] for i in idx[0]]

# ========== TEST ==========
results = search("How is AI used in healthcare?")
print("\n Search Results:")
for r in results:
    print("-", r)

/usr/local/lib/python3.12/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✅ Total vectors stored: 4

 Search Results:
- Text file: Artificial Intelligence is transforming healthcare and education.
- PDF file: This PDF contains information about AI, ML, and Deep Learning.

- Audio file: 
